In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

# import modules
import pandas as pd
import numpy as np

Load and preprocess data

In [3]:
# load excel table as dataframe
df = pd.read_excel(open('iiko.xlsx','rb'))

# delete Unnamed:6 column
del df['Unnamed: 6']

# rename columns
df.columns = ['Date_Time', 'Check', 'Product_group', 'Product', 'Units', 'Sum_before_discount', 'Sum_after_discount', 'Net_cost', 'Markup']

# delete rows containing всего
df = df[~(df.Check.str.contains("всего") == False) ]

# fill NaNs
df = df.fillna(method='ffill')

In [4]:
df.columns

Index([u'Date_Time', u'Check', u'Product_group', u'Product', u'Units',
       u'Sum_before_discount', u'Sum_after_discount', u'Net_cost', u'Markup'],
      dtype='object')

In [5]:
# convert Date_Time to datetime
df['Date_Time'] = pd.to_datetime(df['Date_Time'])
# add columns Year, Months, 
df['Year'] = df['Date_Time'].dt.year
df['Month'] = df['Date_Time'].dt.month
df['Day'] = df['Date_Time'].dt.day
df['Hour'] = df['Date_Time'].dt.hour

# add columns Discount, Profit, Price
df['Discount'] = df['Sum_before_discount'] - df['Sum_after_discount']
df['Profit'] = df['Sum_after_discount']-df['Net_cost']
df['Price'] = df['Sum_after_discount']/df['Units']

# rearrange columns
df = df [['Date_Time', 'Year', 'Month', 'Day', 'Hour', 'Check', 'Product_group', 'Product', 'Units', 'Sum_before_discount', 'Sum_after_discount', 'Net_cost', 'Price', 'Markup', 'Discount', 'Profit']]

In [7]:
#Save data to csv
df.to_csv('iiko.csv',encoding='utf-8')

# Data clearance

We eliminate records for which Units, Sum and Cost are negative

In [ ]:
df = df[(df['Units']>=0) & (df['Net_cost']>=0) & (df['Sum_before_discount']>=0) & (df['Sum_after_discount']>=0)]

# Profitability analysis

Profit = Revenue - Cost
    1. Profit by product and time

Revenue analysis:
    1. revenue by product (product group) and time (month, day, hour)
    2. revenue = #units*price
        2.1 #units by product and time
        2.2 price by product and time
    3. analyze checks to see which products are purchased better together

Cost analysis:
    1. cost by product and time
    2. cost = #units*unit_cost
        2.1 unit_cost by product and time
    3. Breakdown costs by fixed and variable costs

# Profit

In [ ]:
# profit by purchase
df['Profit'].describe()

In [ ]:
# total profit
df['Profit'].sum()

1. Check if there are negative values for profit.

In [ ]:
neg_prof = df[df['Profit']<0]
#neg_prof.head()
neg_prof['Profit'].count()

For some products sum_after_disount is leass then net_cost. Let's check how much of the profit is lacking due to this fact.

In [ ]:
neg_prof['Profit'].sum()

Eleminate this records to proceed with analysis. Calculate profit without disounts.

In [ ]:
#df = df[df['Profit']>=0]
#df['Profit'].sum()

There's 135106/1178133 = 11% of profit is lacking due to discounts. 

Next we breakdown profit by month, days and hours.

In [ ]:
# profit by month
grouping = df.groupby(['Month'])
y = grouping['Profit'].sum()
y.plot(kind='bar')

In [ ]:
# profit by day
for month in range(3,9):
    y = df[df['Month'] == month]
    grouping = y.groupby(['Day'])
    grouping['Profit'].plot(kind='bar')

#profit dynamics
#pr_dyn = (y-y.iloc[1])/y.iloc[1]

In [ ]:
# profit by hours
grouping = df.groupby(['Hour'])
y = grouping['Profit'].sum()
y.plot(kind='bar')

Analysis:
1. In period between 15 and 24 day profit drops by 20%.
2. Profit is twice higher in afternoon (between 15-23) than in the morning and achieves maximum at evening time.

Now we breakdown profit by product groups.

In [ ]:
grouping = df.groupby(['Product_group'])
y = grouping['Profit'].sum() # profit by each group
y.describe()

75% of product groups have profit less than 8000 RUB, which is less than 1% of the total profit (~1200000 RUB). Let's see the profit for top groups.

In [ ]:
top_groups = y[y>8000]
top_groups

Check percentage of profit for each group

In [ ]:
top_groups*100/df['Profit'].sum()

Now analyze major groups by months. Start with ПИВО

In [ ]:
vdf = df[df['Product_group'].str.contains('КОКТЕЙЛИ АЛКОГОЛЬНЫЕ'.decode('utf-8'))]
# profit by day
grouping = vdf.groupby(['Day'])
y = grouping['Profit'].sum()
y.plot(kind='bar')

In [ ]:
#df.boxplot('Profit', by='Product_group')
#plt.yscale('log')

# Revenue

In [ ]:
# Revenue by month
grouping = df.groupby(['Month'])
y = grouping['Sum_after_discount'].sum()
y.plot(kind='bar')

In [ ]:
# Revenue by day
grouping = df.groupby(['Day'])
y = grouping['Sum_after_discount'].sum()
y.plot(kind='bar')

In [ ]:
# Revenue by hours
grouping = df.groupby(['Hour'])
y = grouping['Sum_after_discount'].sum()
y.plot(kind='bar')

Let's take a look at units sold.

In [ ]:
# units by month
grouping = df.groupby(['Month'])
y = grouping['Units'].sum()
y.plot(kind='bar')

In [ ]:
# units by day
grouping = df.groupby(['Day'])
y = grouping['Units'].sum()
y.plot(kind='bar')

In [ ]:
# units by hours
grouping = df.groupby(['Hour'])
y = grouping['Units'].sum()
y.plot(kind='bar')

Analysis:
    1. Revenue follows the same trend as profit
    2. Number of sold units follows the same trend as revenue
    3. Conclusion - profit drops in accordance with the number of units sold.